# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('./data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


In [4]:
data.shape

(68584, 7)

## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [5]:
step1 = data.groupby(['ProductName', 'CustomerID']).agg({'Quantity':'sum'}).sort_values('Quantity', ascending=False).reset_index()
step1.head()

,ProductName,CustomerID,Quantity
0,Longos - Grilled Salmon With Bbq,90069,92
1,Yeast Dry - Fermipan,80694,84
2,Bread - French Baquette,97063,75
3,Spice - Peppercorn Melange,96524,75
4,Fenngreek Seed,97029,75


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [25]:
step2 = step1.pivot_table(index='ProductName', columns='CustomerID', values='Quantity', aggfunc='sum').fillna(0)
step2.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [26]:
step3 = pd.DataFrame(1/(1 + squareform(pdist(step2.T, 'euclidean'))), index=step2.columns, columns=step2.columns)
step3

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [27]:
cust = 356
step4 = list(step3[cust].sort_values(ascending=False)[1:].head().index)
step4

[3317, 1008, 3305, 3253, 2617]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [28]:
step5 = step1.loc[step1['CustomerID'].isin(step4)]
step5

,ProductName,CustomerID,Quantity
58335,Wine - Vidal Icewine Magnotta,3253,2
58371,Cheese - Bocconcini,1008,2
58390,Beans - Wax,3305,2
59577,Table Cloth 81x81 White,3253,2
59848,Pomello,3253,2
...,...,...,...
63599,Curry Paste - Madras,1008,1
63605,Pork - Hock And Feet Attached,3305,1
63607,"Doilies - 5, Paper",3317,1
63621,Pork - Inside,3305,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [29]:
step6 = step5.groupby('ProductName').agg({'Quantity':'sum'}).sort_values('Quantity', ascending=False).reset_index()
step6.head(5)

,ProductName,Quantity
0,"Veal - Inside, Choice",4
1,Wine - Ej Gallo Sierra Valley,3
2,Lamb - Ground,3
3,Wine - Blue Nun Qualitatswein,3
4,Pomello,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [30]:
step7 = pd.merge(step6, pd.DataFrame(step2[cust]), on='ProductName')
step7.columns = ['ProductName', 'Quantity', 'Purchased']
recommend = step7[step7['Purchased'] == 0].head()
recommend


,ProductName,Quantity,Purchased
0,"Veal - Inside, Choice",4,0.0
2,Lamb - Ground,3,0.0
4,Pomello,3,0.0
5,Butter - Unsalted,3,0.0
6,"Tart Shells - Sweet, 4",3,0.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [32]:
step8 = list(data['CustomerID'].unique())
recommend = {}

for x in step8:
    similar_cust = list(step3[x].sort_values(ascending=False)[1:].head().index)
    sim_cust_prod = step1[step1['CustomerID'].isin(similar_cust)]
    grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
    ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
    
    merged = pd.merge(ranked_products, pd.DataFrame(step2[x]), on='ProductName')
    merged.columns = ['ProductName', 'Quantity', 'Purchased']
    recs = merged[merged['Purchased']==0].head()
    recommend[x] = list(recs['ProductName'])

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [34]:
step9 = pd.DataFrame.from_dict(recommend, orient='index').reset_index()
step9.columns = ['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
step9

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,61288,Jagermeister,Chicken - Soup Base,Lime Cordial - Roses,Macaroons - Two Bite Choc,Flavouring - Orange
1,77352,Wine - Valpolicella Masi,Tahini Paste,Tuna - Salad Premix,Chicken - Soup Base,Isomalt
2,40094,Bread - Italian Corn Meal Poly,Puree - Mocha,Tuna - Salad Premix,Sherry - Dry,Beer - Sleemans Cream Ale
3,23548,Squid - Tubes / Tenticles 10/20,Sprouts - Baby Pea Tendrils,Flavouring - Orange,Bread - Raisin Walnut Oval,Banana Turning
4,78981,Vanilla Beans,Lettuce - Frisee,"Yogurt - Blueberry, 175 Gr",Cinnamon Buns Sticky,Pop Shoppe Cream Soda
...,...,...,...,...,...,...
995,49005,"Pasta - Penne, Rigate, Dry",Browning Caramel Glace,Tea - Decaf Lipton,"Garlic - Primerba, Paste","Wine - Red, Harrow Estates, Cab"
996,41286,Veal - Inside,"Thyme - Lemon, Fresh",Wonton Wrappers,"Veal - Inside, Choice",Bread Crumbs - Japanese Style
997,85878,Olives - Kalamata,Wine - White Cab Sauv.on,Veal - Eye Of Round,Bread - Calabrese Baguette,Cheese - Wine
998,68506,"Veal - Brisket, Provimi,bnls",Beef - Ground Medium,"Rosemary - Primerba, Paste",Scallops - 10/20,"Oregano - Dry, Rubbed"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [37]:
step10 = pd.DataFrame(1/(1 + squareform(pdist(step2.T, 'jaccard'))), index=step2.columns, columns=step2.columns)
step8 = list(data['CustomerID'].unique())
recommend10 = {}

for x in step8:
    similar_cust = list(step10[x].sort_values(ascending=False)[1:].head().index)
    sim_cust_prod = step1[step1['CustomerID'].isin(similar_cust)]
    grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
    ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
    
    merged = pd.merge(ranked_products, pd.DataFrame(step2[x]), on='ProductName')
    merged.columns = ['ProductName', 'Quantity', 'Purchased']
    recs = merged[merged['Purchased']==0].head()
    recommend10[x] = list(recs['ProductName'])

step10_res = pd.DataFrame.from_dict(recommend10, orient='index').reset_index()
step10_res.columns = ['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
step10_res

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,61288,"Cheese - Brie, Triple Creme",V8 - Berry Blend,Ecolab - Mikroklene 4/4 L,Flavouring - Orange,Mustard - Seed
1,77352,Cheese - Wine,Sea Bass - Whole,Garlic,Beef - Top Sirloin - Aaa,Beer - Blue
2,40094,"Juice - Cranberry, 341 Ml",Rosemary - Dry,"Wine - Red, Colio Cabernet",Sponge Cake Mix - Chocolate,Pasta - Angel Hair
3,23548,Wine - Hardys Bankside Shiraz,Table Cloth 81x81 White,Cookies Cereal Nut,Longos - Chicken Wings,"Wine - Cahors Ac 2000, Clos"
4,78981,Muffin Batt - Blueberry Passion,Pie Filling - Cherry,Cocoa Butter,Wine - Blue Nun Qualitatswein,Cheese - Cottage Cheese
...,...,...,...,...,...,...
995,49005,Juice - Happy Planet,Lettuce - Frisee,Eggplant - Asian,Smoked Paprika,Mussels - Frozen
996,41286,Wine - Redchard Merritt,Towels - Paper / Kraft,Hot Chocolate - Individual,Berry Brulee,"Lamb - Pieces, Diced"
997,85878,Scampi Tail,Lettuce - Treviso,Zucchini - Yellow,Bread Fig And Almond,Pastry - Cheese Baked Scones
998,68506,"Cup - 6oz, Foam","Coconut - Shredded, Sweet","Wine - Cahors Ac 2000, Clos",Rice - Jasmine Sented,"Soup - Canadian Pea, Dry Mix"
